## 데이터 로드

In [1]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [2]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date'], axis=1)
train_data.head()

,tags,songs,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [4]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [5]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [6]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [7]:
train_data_sample = train_data[500:1000].copy()
train_data_sample = train_data_sample.reset_index(drop=True)
#train_data_sample = train_data

# 한국어 -> 영어 변환
- 한국어 전처리에 어려움이 있어서, 영어로 모두 변환 후 처리

In [11]:
from googletrans import Translator
import time

def trans_to_eng(tags):
    time.sleep(1)
    translator = Translator()
    translated = translator.translate(tags, src = 'ko', dest = 'en')
    return translated.text

for i in range(len(train_data_sample)):
    eng_tags = [trans_to_eng(tag) for tag in train_data_sample['tags'][i]]
    train_data_sample['tags'][i] = eng_tags

train_data_sample.to_json('../Datasets/train_eng1.json', orient='records')
train_data_sample.head(30)

    
#train_data.head()
#eng_tags = [trans_to_eng(tag) for tag in train_data['tags'][1]]
#print("Kor: ", train_data['tags'][1])
#print("Eng: ", eng_tags)

AttributeError: 'Translator' object has no attribute 'raise_Exception'

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [171]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head(30)

,tags,song_id,like_cnt
0,"[빌로우, EDMFloor]",582347,0
1,"[빌로우, EDMFloor]",577988,0
2,"[빌로우, EDMFloor]",4096,0
3,"[빌로우, EDMFloor]",223252,0
4,"[빌로우, EDMFloor]",108282,0
5,"[빌로우, EDMFloor]",279668,0
6,"[빌로우, EDMFloor]",419946,0
7,"[빌로우, EDMFloor]",430613,0
8,"[빌로우, EDMFloor]",57445,0
9,"[빌로우, EDMFloor]",302788,0


In [172]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'히트곡', '댄스', '잔잔한', '회상', '걸그룹', '추억', 'kpop', '아이돌'}


In [173]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(18275, 3)

In [174]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

C:\Users\pigwo\AppData\Local\Temp\ipykernel_1872\1966443478.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sample['tags'].iloc[i] = list(train_dict[song])


,tags,song_id,like_cnt
0,"[빌로우, EDMFloor]",582347,0
1,"[빌로우, EDMFloor]",577988,0
2,"[빌로우, EDMFloor]",4096,0
3,"[빌로우, EDMFloor]",223252,0
4,"[클럽, 스트레스, 빌로우, EDMFloor]",108282,0


In [175]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,like_cnt,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,"[빌로우, EDMFloor]",582347,0,"[GN1104, GN1103, GN1101]",20171220,10121025,[784873],No Coming Down,[Nghtmre]
1,"[빌로우, EDMFloor]",577988,0,"[GN1014, GN1001]",20180605,10135634,[718938],Get Out,[CHVRCHES]
2,"[빌로우, EDMFloor]",4096,0,"[GN1014, GN1001]",20180605,10135634,[718938],Miracle,[CHVRCHES]
3,"[빌로우, EDMFloor]",223252,0,"[GN2701, GN2702, GN2703, GN1104, GN1103, GN1101]",20180413,10157297,[990703],Batshit,[Sofi Tukker]
4,"[클럽, 스트레스, 빌로우, EDMFloor]",108282,0,"[GN2701, GN2702, GN2703, GN1104, GN1103, GN1101]",20180413,10157297,[990703],"Best Friend (Feat. NERVO, The Knocks & Alisa U...",[Sofi Tukker]


In [176]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18275 entries, 0 to 18274
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                18275 non-null  object
 1   song_id             18275 non-null  int64 
 2   like_cnt            18275 non-null  int64 
 3   gnr                 18275 non-null  object
 4   issue_date          18275 non-null  int64 
 5   album_id            18275 non-null  int64 
 6   artist_id_basket    18275 non-null  object
 7   song_name           18275 non-null  object
 8   artist_name_basket  18275 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.4+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [177]:
train_data_sample2 = train_data[500:1000].copy()
train_data_sample2 = train_data_sample2.reset_index(drop=True)
#train_data_sample2 = train_data

In [178]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 1, workers = 4, sg = 1)

w2v.wv.vectors.shape

(689, 100)

In [179]:
print(w2v.wv.most_similar('휴식'))
print(w2v.wv.most_similar('락'))

[('수능금지곡', 0.31057876348495483), ('썸', 0.27234721183776855), ('설렘', 0.25885286927223206), ('힘내', 0.23712459206581116), ('멜랑꼴리', 0.2330673635005951), ('볼빨간사춘기', 0.22802220284938812), ('잠', 0.22291728854179382), ('쓸쓸', 0.22209863364696503), ('도시의밤', 0.21834705770015717), ('0살', 0.2179148942232132)]
[('스쿨오브록', 0.31354519724845886), ('바다', 0.3078239858150482), ('밤', 0.2771238684654236), ('기분좋은', 0.24497230350971222), ('오후', 0.23881928622722626), ('생각나는', 0.23669685423374176), ('쉼', 0.23351064324378967), ('페스티벌', 0.23336388170719147), ('외국힙합', 0.23205439746379852), ('드라마', 0.23142565786838531)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [180]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(18275, 197)

In [181]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [182]:
simi_test = gnr_sim[1, :]
simi_test2 = gnr_sim[2, :]
print(simi_test + simi_test2)
print(song_data[song_data['song_id']==525514])
print(song_data[song_data['song_id']==129701])
print(song_data[song_data['song_id']==229622])

[0. 2. 2. ... 0. 0. 0.]
                     gnr  issue_date  album_id artist_id_basket  \
525514  [GN1402, GN1401]    20130506   2200223         [734201]   

              song_name artist_name_basket  song_id  
525514  Hey Little Girl          [The Sol]   525514  
                             gnr  issue_date  album_id artist_id_basket  \
129701  [GN0901, GN0902, GN1001]    20130917   2201802         [536907]   

       song_name artist_name_basket  song_id  
129701   Octagon      [Royal Bangs]   129701  
                                     gnr  issue_date  album_id  \
229622  [GN1402, GN1401, GN0901, GN0902]    20061212    528351   

       artist_id_basket  song_name artist_name_basket  song_id  
229622         [406781]  Say Hello     [Rosie Thomas]   229622  


- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [183]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_cv(songs):
    songs['gnr_literal'] = songs['gnr'].apply(lambda x : (' ').join(x))
    count_vect = CountVectorizer()
    gnr_mat = count_vect.fit_transform(songs['gnr_literal'])
    return gnr_mat

def get_sim(song_index, gnr_mat, sim):
    if sim == 'cos':
        gnr_sim = cosine_similarity(gnr_mat[song_index], gnr_mat)
    
    elif sim == 'jac':
        gnr_sim = np.zeros(gnr_mat.shape[0])
        
        song_set = set(gnr_mat[song_index].indices)
    
        for i in range(gnr_mat.shape[0]):
            other_set = set(gnr_mat[i].indices)
            
            union = song_set.union(other_set)
            intersection = song_set.intersection(other_set)
            
            if len(union) != 0:
                gnr_sim[i] = len(intersection)/len(union)
                
    elif sim == 'pea':
        gnr_sim = np.zeros(gnr_mat.shape[0])
    
        a = gnr_mat[song_index].toarray()
    
        for i in range(gnr_mat.shape[0]):
            b = gnr_mat[i].toarray()
            b = b.T
        
            gnr_sim[i] = np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

    return gnr_sim

In [184]:
import numpy as np

def apply_genre_weight(mat):
    genre_nums = mat.getnnz(0)
    genre_num_max = max(genre_nums)
    genre_weight = np.log(genre_num_max / genre_nums + 1)
    
    result_mat = mat.copy()
    
    for i in range(result_mat.shape[0]):
        for j in result_mat[i].indices:
            result_mat[(i, j)] *= genre_weight[j]
    
    return result_mat

weight_mat = apply_genre_weight(get_cv(song_tag_appended))
print(weight_mat)

  (0, 70)	1
  (0, 69)	2
  (0, 67)	1
  (1, 66)	4
  (1, 53)	1
  (2, 66)	4
  (2, 53)	1
  (3, 70)	1
  (3, 69)	2
  (3, 67)	1
  (3, 187)	3
  (3, 188)	4
  (3, 189)	3
  (4, 70)	1
  (4, 69)	2
  (4, 67)	1
  (4, 187)	3
  (4, 188)	4
  (4, 189)	3
  (5, 70)	1
  (5, 67)	1
  (5, 187)	3
  (5, 189)	3
  (5, 48)	4
  (5, 190)	2
  :	:
  (18256, 121)	1
  (18256, 126)	4
  (18257, 121)	1
  (18258, 121)	1
  (18259, 121)	1
  (18260, 121)	1
  (18261, 121)	1
  (18262, 121)	1
  (18263, 121)	1
  (18264, 121)	1
  (18265, 121)	1
  (18265, 129)	3
  (18266, 121)	1
  (18267, 121)	1
  (18268, 121)	1
  (18269, 67)	1
  (18270, 121)	1
  (18271, 121)	1
  (18271, 129)	3
  (18272, 121)	1
  (18272, 129)	3
  (18273, 121)	1
  (18273, 129)	3
  (18274, 121)	1
  (18274, 129)	3


In [185]:
def find_sim_song(df, sim, mat, songs, genre_imb_mode=False, top_n=0,):
    simi = np.zeros(len(df['song_id']))
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
            
        if genre_imb_mode:
            sim_array = get_sim(title_index, weight_mat, sim)
        
        else:
            sim_array = get_sim(title_index, mat, sim)
            
        simi = simi + sim_array
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    
    # for song in songs:
    #     title_song = df[df['song_id'] == song]
    #     title_index = title_song.index.values
        
    #     temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
        
    # 유사도가 0.5 이하인 경우는 제외
    #temp = temp[temp['similarity'] >= 0.5]
    if top_n < 1:
        temp = temp[temp['similarity'] >= 0.4]
        temp = temp.reset_index(drop=True)
        return temp
    else:
        temp = temp.reset_index(drop=True)
        return temp.iloc[ : top_n]
    
    # final_index = temp.index.values[ : top_n]
    

In [186]:
'''
similar_songs = find_sim_song(song_tag_appended, 'cos', gnr_mat, [525514, 129701, 229622], 10)

similar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]
'''

"\nsimilar_songs = find_sim_song(song_tag_appended, 'cos', gnr_mat, [525514, 129701, 229622], 10)\n\nsimilar_songs[['song_id', 'similarity', 'issue_date', 'gnr']]\n"

## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [187]:
'''
def song_recommend(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
        print(simi_songs)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 태그와 유사도 두 부분에서 동일한 개수(홀수일 경우 태그 > 유사도) 추출
        # sc = 유사도 부분에서 추출할 노래의 개수
        if len(recommended) % 2 == 0:
            sc = (10-len(recommended)) / 2
        else:
            sc = (10-len(recommended)) // 2
        
        # 태그는 있고 히스토리가 없는 경우 태그 부분에서 10개를 추출하기 위한 코드
        if len(songs) == 0:
            sc = 0
        
        # 이미 추출한 노래를 제외하고 태그 부분에서 정해진 개수만큼 추출한다
        # 태그가 없을 경우 동작하지 않음
        index = 0
        while len(tag_songs) != 0 and len(recommended) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommended:
                recommended.append(tag_song)
            
            index += 1
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]
'''

"\ndef song_recommend(tags, songs, tag_df, song_df, sim_mat):\n    ts = tags\n    \n    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다\n    all_tags = []\n    if len(ts) != 0 and len(ts) < 3:\n        for tag in ts:\n            sim_tags = w2v.wv.most_similar(tag)\n            for t in sim_tags:\n                all_tags.append(t)\n        all_tags = sorted(all_tags, key = lambda x : -x[1])\n        i = 0\n        while len(ts) != 0 and len(ts) < 3:\n            tag = all_tags[i][0]\n            if tag not in ts:\n                ts.append(tag)\n                i += 1\n\n    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다\n    tag_songs = dict()\n    \n    for tag in ts:\n        for i in range(len(tag_df['song_id'])):\n            if tag in tag_df['tags'][i]:\n                \n                for ss in tag_df['song_id'][i]:\n                    if not ss in songs:\n                        \n                        if ss in tag_songs:\n                            tag_songs[ss] += 1\n  

In [188]:
'''
def song_recommend_test(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1

    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    index = 0
    
    while len(recommended) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommended.append(tag_song)
            
        index += 1
    
    
    
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommended) < 10:
        
        # 이미 추출한 노래를 제외하고 추천 노래가 10개가 될떄까지
        # 유사도 부분에서 추출한다
        index = 0
        while len(recommended) < 10 and len(simi_songs) != 0 and len(simi_songs) > index:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommended:
                recommended.append(simi_song)
            
            index += 1
            
        if len(recommended) < 10:
            index = 0
            while len(tag_songs) != 0 and len(recommended) < 10:
                tag_song = tag_songs[index][0]
                if not tag_song in recommended:
                    recommended.append(tag_song)
                index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommended:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0])
    
    return song_df.iloc[rec_index]
'''

"\ndef song_recommend_test(tags, songs, tag_df, song_df, sim_mat):\n    ts = tags\n    \n    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다\n    all_tags = []\n    if len(ts) != 0 and len(ts) < 3:\n        for tag in ts:\n            sim_tags = w2v.wv.most_similar(tag)\n            for t in sim_tags:\n                all_tags.append(t)\n        all_tags = sorted(all_tags, key = lambda x : -x[1])\n        i = 0\n        while len(ts) != 0 and len(ts) < 3:\n            tag = all_tags[i][0]\n            if tag not in ts:\n                ts.append(tag)\n                i += 1\n\n    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다\n    tag_songs = dict()\n    \n    for tag in ts:\n        for i in range(len(tag_df['song_id'])):\n            if tag in tag_df['tags'][i]:\n                \n                for ss in tag_df['song_id'][i]:\n                    if not ss in songs:\n                        \n                        if ss in tag_songs:\n                            tag_songs[ss] += 

# 모델 평가
- [참고](https://chrisjune-13837.medium.com/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-%EC%84%B1%EB%8A%A5%ED%8F%89%EA%B0%80%EB%B0%A9%EB%B2%95-with-python-9932097f0ff9)

# Recall@K

In [189]:
def get_recall_k(y_true, y_pred):
    recall_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(true_items) if len(true_items) > 0 else 0
    return recall

#recall_k = get_recall_k(my_songs1, pred_list)
#print("Recall@K (K=10): {:.2f}".format(recall_k))

# Precision@K

In [190]:
def get_precision_k(y_true, y_pred):
    precision_k = 0
    
    true_items = set(y_true)
    pred_items = set(y_pred)
    intersect_items = len(true_items.intersection(pred_items))
    recall = intersect_items / len(pred_items) if len(pred_items) > 0 else 0
    return recall

#precision_k = get_precision_k(my_songs1, pred_list)
#print("Precision@K (K=10): {:.2f}".format(precision_k))

# Average Precision@K
- 아이템 개수가 매우 적은 경우, 모델의 성능 또한 낮아지는 문제가 발생
    - 해결방법
        1. Thresholding : 사용자 플레이리스트 내 음악이 일정 개수 이상인 경우만 적용
        2. Item Weighting : hits 에 1이 아닌 가중치를 이용. 이 때 precision 이 1 보다 크지 않게 하기 위해, 정규화 필요

In [191]:
def get_ap_k(y_true, y_pred, k):
    pred_items = y_pred[:k]
    hits = []
    for item in pred_items:
        if item in y_true:
            hits.append(1)
        else:
            hits.append(0)
    precision_values = []
    for i in range(1, k+1):
        precision_values.append(sum(hits[:i]) / i)
        
    print(precision_values)
    
    if len(precision_values) == 0:
        return 0
    else:
        return sum(precision_values) / len(precision_values)
    
#ap_k = get_ap_k(my_songs1, pred_list, 10)
#print("AP@K (K=10): {:.2f}".format(ap_k))

# Mean Average Precision@K

In [192]:
def get_map_k(y_true, y_pred, k):
    sum_ap = 0
    for true_item, pred_item in zip(y_true, y_pred):
        ap_k = get_ap_k(true_item, pred_item, k)
        sum_ap += ap_k
    if len(y_true) == 0:
        return 0
    else:
        return sum_ap / len(y_true)
    
#map_k = get_map_k(all_my_songs, all_pred_songs, 10)
#print("MAP@K (K=10): {:.2f}".format(map_k))

In [193]:
train_data.head(10)

,tags,song_id,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9
5,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[418694, 222305, 96545, 135950, 304687, 457451...",435
6,"[짝사랑, 취향저격, 슬픔, 고백, 사랑, 이별]","[528130, 505036, 398947, 627363, 37748, 376358...",24
7,"[잔잔한, 추억, 회상]","[496913, 632529, 501426, 515574, 411161, 10341...",5
8,"[일렉트로니카, 포크, 메탈, 락, 댄스, 인디]","[634861, 270738, 163936, 692209, 449477, 56342...",0
9,"[록, Metal, 이일우, M에센셜, 메탈, Rock, 락]","[613315, 439294, 230806, 497932, 113479, 93506...",7


# 추천 함수 수정

### 사용자 플레이리스트 내 음악의 태그를 포함하는 노래 중에서 장르 유사도를 측정

In [194]:
def song_recommend(tags, songs, tag_df, song_df, sim):
    
    ts = tags
    '''
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1
    '''
                
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        #print("현재 태그: {}".format(tag))
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                #print("태그에 맞는 노래: {}".format(tag))
                for ss in tag_df['song_id'][i]:
                    #print(ss)
                    if ss in tag_songs:
                        tag_songs[ss] += 1
                            
                    else:
                        tag_songs[ss] = 1
                        
    #tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    tag_songs = {key: value for key, value in tag_songs.items() if value > 0}
    tag_songs = list(tag_songs)
    tag_songs = song_df[song_df['song_id'].isin(tag_songs)]
    tag_songs = tag_songs.reset_index(drop=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(tag_songs, sim, get_cv(tag_songs), songs, False, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    
    for rec in simi_songs['song_id']:
        title_song = tag_songs[tag_songs['song_id'] == rec]
        title_index = title_song.index
        recommended.append(title_index[0])
    
    return tag_songs.iloc[recommended[:10]]



### 사용자 플레이리스트 내 음악과 높은 유사도를 가진 곡들 중, 태그 유사도가 높은 곡을 추천
- 높은 유사도를 가진 곡을 상위 100개로 할지, 일정 유사도보다 높은 경우로 할지를 나눠서 테스트할 예정
- 태그에 가중치를 부여할 때, 추출된 음악들 중에서가 아닌 사용자 플레이리스트 내 음악들에 대해 부여할 방법 고려 중

In [195]:
def make_song_num_dict(data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(data)):
        songs = data['song_id'][i]
        tags = data['tags'][i]
        
        for j in tags:
            if not j in song_ids:
                song_ids[j] = set(songs)
            
            else:
                song_ids[j].update(songs)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

song_num_dict, song_num_max = make_song_num_dict(train_data_sample2)
tag_weights = {tag: np.log(song_num_max / cnt + 1) for tag, cnt in song_num_dict.items()}
print(tag_weights)


{'빌로우': 4.34287734754113, 'EDMFloor': 4.34287734754113, '기분전환': 0.6931471805599453, '밤': 1.1911757529923772, '새벽': 1.051397065763938, '댄스': 1.5401987051902684, '걸그룹': 2.3723500240898385, '발라드': 0.937697392511506, '휴식': 0.9790037211447301, '가을': 1.819465023950707, '잔잔한': 0.9239790617108892, '귀르가즘': 5.299878647114989, '우울': 2.770826607246382, 'soul': 3.374895805767733, 'JAZZY': 5.299878647114989, '감성': 1.1093889381298705, 'groove': 5.299878647114989, 'OST': 1.6629405606477428, '신곡': 4.611711246531472, '박자': 4.611711246531472, '비트': 3.5741149559120613, '재즈': 1.7671049076641145, '월요병': 3.3921475461422377, '좋은노래': 5.239565989113283, '음색': 3.349601561179422, '힐링': 1.106407545670478, '편안한': 2.966999719470782, '겨울': 1.7302246807883075, '저녁': 2.041165760089332, '매장음악': 1.4115088077314495, '겨울감성': 3.0472142293891347, '카페': 1.3177455223761656, '드라이브': 0.8164572733184746, '사랑': 1.0324094960215011, '피크닉': 4.817486098650238, '오후': 2.7153977727326044, '여행': 1.6086857499315461, '운동': 1.871536889726670

In [196]:
def get_tag_simi(input_tag, simi_tags, imb_mode):
    if imb_mode == False:
        simi_lst = [w2v.wv.similarity(input_tag, simi_tag) for simi_tag in simi_tags]
        return max(simi_lst)
    else:
        weighted_simi_lst = []
        for tag in simi_tags:
            similarity = w2v.wv.similarity(input_tag, tag)
            if similarity < 0:
                weighted_simi_lst.append(tag_weights.get(tag, 0)/similarity)
            else:
                weighted_simi_lst.append(tag_weights.get(tag, 0)*similarity)
        return max(weighted_simi_lst)
        
def song_recommend2(tags, songs, tag_df, song_df, sim, tag_imb_mode, genre_imb_mode):
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        # 장르 불균형데이터에 대해서는 상위 100개의 음악을 추출하는 것이 유효함
        #if genre_imb_mode:
        simi_songs = find_sim_song(song_df, sim, get_cv(song_df), songs, genre_imb_mode, 100)
            
        #else:
        #simi_songs = find_sim_song(song_df, sim, get_cv(song_df), songs) #상위 100개가 아닌, 유사도가 0.4 이상인 음악만 추출
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    #print(simi_songs)
    
    '''

    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 3개까지 늘린다
    all_tags = []
    if len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            sim_tags = w2v.wv.most_similar(tag)
            for t in sim_tags:
                all_tags.append(t)
        all_tags = sorted(all_tags, key = lambda x : -x[1])
        i = 0
        while len(ts) != 0 and len(ts) < 3:
            tag = all_tags[i][0]
            if tag not in ts:
                ts.append(tag)
                i += 1
    '''
       
    ts = tags
    
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    tag_simi_mean = []
    
    for simi_tags in simi_songs['tags']:
        songs_similality = []
        for input_tag in ts:
            songs_similality.append(get_tag_simi(input_tag, simi_tags, tag_imb_mode))
        tag_simi_mean.append(sum(songs_similality) / len(songs_similality))
    
    tag_simi_mean = pd.DataFrame(columns = ['tag_simi'], data = tag_simi_mean)
    sorted_idx = tag_simi_mean['tag_simi'].sort_values(ascending = False).index
    #for i in sorted_idx:
    #    print("원본: {} / simi_mean: {}".format(simi_songs.loc[i, 'song_id'], tag_simi_mean.loc[i, 'tag_simi']))
    tag_simi_mean = simi_songs.loc[sorted_idx]
    
    '''
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommended = []
    
    for rec in simi_songs['song_id']:
        title_song = tag_songs[tag_songs['song_id'] == rec]
        title_index = title_song.index
        recommended.append(title_index[0])
    
    '''
    
    return tag_simi_mean.iloc[:10]



In [197]:
test_data_sample = train_data[500:1000].copy()
test_data_sample = test_data_sample.reset_index(drop=True)
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

rec1 = song_recommend(test_my_tags[0], test_my_songs[0], train_data_sample2, song_tag_appended, 'cos')

pred_list = rec1['song_id'].tolist()
precision_k = get_precision_k(test_my_songs[0], pred_list)
recall_k = get_recall_k(test_my_songs[0], pred_list)

print("Recall@K (K=10): {:.2f}".format(recall_k))
print("Precision@K (K=10): {:.2f}".format(precision_k))

Recall@K (K=10): 0.24
Precision@K (K=10): 1.00


In [198]:
'''
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend(tags, plist, train_data_sample2, song_tag_appended, 'cos')
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])
'''

"\npred_list = []\nfor plist, tags in zip(test_my_songs, test_my_tags):\n    recommended = song_recommend(tags, plist, train_data_sample2, song_tag_appended, 'cos')\n    pred_list.append(recommended['song_id'].tolist())\nprint(pred_list[0])\n"

In [199]:
'''
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))
'''

'\nmap_k = get_map_k(test_my_songs, pred_list, 10)\nprint("MAP@K (K=10): {:.2f}".format(map_k))\n'

# 태그, 장르 불균형 데이터 처리 X

In [200]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend2(tags, plist, train_data_sample2, song_tag_appended, 'cos', False, False)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[118237, 460096, 192605, 488758, 372459, 433394, 652086, 650879, 488634, 471405]


In [201]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.3333333333333333, 0.25, 0.4, 0.5, 0.5714285714285714, 0.625, 0.6666666666666666, 0.7]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.75, 0.6, 0.5, 0.42857142857142855, 0.375, 0.3333333333333333, 0.3]
[0.0, 0.0, 0.3333333333333333, 0.5, 0.6, 0.5, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[1.0, 1.0, 0.6666666666666666, 0.5, 0.4, 0.3333333333333333, 0.2857142857142857, 0.25, 0.2222222222222222, 0.2]
[0.0, 0.5, 0.6666666666666666, 0.5, 0.4, 0.3333333333333333, 0.42857142857142855, 0.375, 0.3333333333333333, 0.3]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 0.7142857142857143, 0.625, 0.5555555555555556, 0.5]
[1.0, 1.0, 0.6666666666666666,

# 태그 불균형 데이터 처리

In [202]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend2(tags, plist, train_data_sample2, song_tag_appended, 'cos', True, False)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[118237, 488758, 460096, 192605, 652086, 372459, 433394, 322072, 281488, 251363]


In [203]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.2, 0.3333333333333333, 0.2857142857142857, 0.375, 0.4444444444444444, 0.5]
[1.0, 1.0, 0.6666666666666666, 0.75, 0.8, 0.8333333333333334, 0.8571428571428571, 0.875, 0.8888888888888888, 0.9]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 0.75, 0.6, 0.5, 0.42857142857142855, 0.375, 0.3333333333333333, 0.3]
[0.0, 0.5, 0.6666666666666666, 0.5, 0.6, 0.5, 0.5714285714285714, 0.625, 0.5555555555555556, 0.5]
[1.0, 1.0, 0.6666666666666666, 0.5, 0.4, 0.3333333333333333, 0.2857142857142857, 0.25, 0.2222222222222222, 0.2]
[0.0, 0.5, 0.6666666666666666, 0.5, 0.4, 0.5, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.8333333333333334, 0.7142857142857143, 0.625, 0.5555555

# 장르 불균형 데이터 처리

In [204]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend2(tags, plist, train_data_sample2, song_tag_appended, 'cos', False, True)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[118237, 488758, 192605, 460096, 322880, 96726, 514157, 229482, 23768, 229112]


In [205]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.5, 0.6666666666666666, 0.75, 0.8, 0.8333333333333334, 0.8571428571428571, 0.875, 0.7777777777777778, 0.7]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.875, 0.8888888888888888, 0.9]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8]
[0.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.5, 0.3333333333333333, 0.5, 0.6, 0.6666666666666666, 0.5714285714285714, 0.5, 0.5555555555555556, 0.6]
[0.0, 0.5, 0

# 태그, 장르 불균형 데이터 처리

In [206]:
pred_list = []
for plist, tags in zip(test_my_songs, test_my_tags):
    recommended = song_recommend2(tags, plist, train_data_sample2, song_tag_appended, 'cos', True, True)
    pred_list.append(recommended['song_id'].tolist())
print(pred_list[0])

[118237, 488758, 192605, 460096, 322880, 514157, 23768, 96726, 440373, 322072]


In [207]:
map_k = get_map_k(test_my_songs, pred_list, 10)
print("MAP@K (K=10): {:.2f}".format(map_k))

[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.5, 0.6666666666666666, 0.75, 0.6, 0.6666666666666666, 0.7142857142857143, 0.75, 0.7777777777777778, 0.8]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8888888888888888, 0.8]
[0.0, 0.5, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.125, 0.1111111111111111, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.3333333333333333, 0.25, 0.2, 0.16666666666666666, 0.14285714285714285, 0.25, 0.2222222222222222, 0.2]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 1.0, 1.0, 0.8, 0.6666666666666666, 0.5714285714285714, 0.5, 0.4444444444444444, 0.4]
[1.0, 1.0, 0.6666666666666666, 0.75, 0.6, 0.5, 0.42857142857142855, 0.5, 0.5555555555555556,

# nDCG

In [208]:
def ndcg_at_k(y_true, y_pred, k):
    ndcg = 0
    ranking = []
    
    for i in range(k):
        if y_pred[i] in y_true:
            ranking.append(1)
        else:
            ranking.append(0.1)
    print(ranking)
        
    # Ideal ranking을 계산하기 위해 ranking을 내림차순으로 정렬한 별도의 리스트
    ideal_ranking = sorted(ranking, reverse=True)
    
    # DCG 계산
    dcg = ranking[0]
    for i in range(1, min(k, len(ranking))):
        dcg += ranking[i] / np.log2(i + 1)
    
    # Ideal DCG 계산
    ideal_dcg = ideal_ranking[0]
    for i in range(1, min(k, len(ideal_ranking))):
        ideal_dcg += ideal_ranking[i] / np.log2(i + 1)
    
    # nDCG 계산
    if ideal_dcg == 0:
        ndcg = 0
    else:
        ndcg = dcg / ideal_dcg
    
    return ndcg

In [209]:
ndcg = 0
for true_item, pred_item in zip(test_my_songs, pred_list):
    ndcg += ndcg_at_k(true_item, pred_item, k=10)
    
ndcg_result = ndcg / len(test_my_songs)
print(f'nDCG: {ndcg_result:.4f}')

[1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 0.1, 1, 1, 0.1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 1, 1, 0.1, 0.1]
[0.1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 1, 0.1, 0.1, 0.1, 0.1, 1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 0.1, 1, 0.1, 0.1, 0.1, 1, 1, 1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1, 1, 0.1, 0.1]
[0.1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0.1]
[1, 1, 1, 1, 1, 1, 0.1, 0.1, 0

## 태그 세트 생성

In [210]:
def make_tag_set(data):
    tag_set = set()

    for i in range(len(data)):
        tag = data['tags'][i]
    
        for j in tag:
            tag_set.add(j)

    return tag_set

In [211]:
tag_set = make_tag_set(train_data_sample2)
print(tag_set)

{'61회', 'KPOP탐구생활', '조용한', '플레이리스트', '인디뮤직', '찬양', '잔잔한노래', '운전', '산책이나', '방탄소년단', '팝', '7080', '도시의밤', '밤에듣기좋은노래', 'hilite', '무한도전', '꿀보이스', '도시', '베르디', '90년대', '게임', '차트', '집콕', '하트시그널3', '아재', '비', '홍연언니', '편집샵', '가사없는', '외로운', '기분을', '피쉬슈즈', '중독성', '한국ost', '일본', '화이팅', '여성', '외로움', '5살', '여가수가많타', '스웨그', '자전거', '봄바람', '잠자기_전', '장르Top100', '홈트레이닝', 'aomg', '8090세대', 'TR808', '루프탑', '기분좋은', '대화', '생각나는', '피크닉', '운동', '외힙입문', '스테디셀러', '노래방', '리듬앤블루스', '음색부자', '추천노래', '장르불문', '프랑스', '독서', '도입부부터좋은', 'Motown', '응답하라', '팝발라드', '벚꽃', '잔잔하고_따스한', '아이돌', '어번컨템포러리', '아픔', 'TOP100', '트로트Top100', '걸그룹', '아이돌탐구생활', '잔잔한_노래', '선선한밤', '세련된', '새벽', '썸', '성인가요Top100', '미국', '국내힙합', '힙합클럽', '혼자', '러블리쏘', '노을', '썸녀', '머리를맑게해주는', '왜그럴까', 'composer', '부드럽고', 'OST', '4살', '연주곡', '가을감성', '일본노래추천', '빌보드Hot50', '성탄절', '나만의Best3', '달콤', '도입부', '비트', '원곡', '인기곡', '헤어진후', '그리움', '식케이', '공감', '위로', 'Pop', '겨울공기', 'Rock', '겨울감성', '감미로움', '명상', '신청곡', '아기클래식', '좋은노래', '많이들어본', '데이트', '국내', '아기동화', '운전할때', '노동요

## 태그 dict 생성 함수

## 영어 대소문자 통일

- 소문자로 통일

In [212]:
def make_dict_lower(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            tag_dict[i] = i.lower()
    
    return tag_dict

In [213]:
tag_lower_dict = make_dict_lower(tag_set)
print(tag_lower_dict)

{'61회': '61회', 'KPOP탐구생활': 'kpop탐구생활', '조용한': '조용한', '플레이리스트': '플레이리스트', '인디뮤직': '인디뮤직', '찬양': '찬양', '잔잔한노래': '잔잔한노래', '운전': '운전', '산책이나': '산책이나', '방탄소년단': '방탄소년단', '팝': '팝', '7080': '7080', '도시의밤': '도시의밤', '밤에듣기좋은노래': '밤에듣기좋은노래', 'hilite': 'hilite', '무한도전': '무한도전', '꿀보이스': '꿀보이스', '도시': '도시', '베르디': '베르디', '90년대': '90년대', '게임': '게임', '차트': '차트', '집콕': '집콕', '하트시그널3': '하트시그널3', '아재': '아재', '비': '비', '홍연언니': '홍연언니', '편집샵': '편집샵', '가사없는': '가사없는', '외로운': '외로운', '기분을': '기분을', '피쉬슈즈': '피쉬슈즈', '중독성': '중독성', '한국ost': '한국ost', '일본': '일본', '화이팅': '화이팅', '여성': '여성', '외로움': '외로움', '5살': '5살', '여가수가많타': '여가수가많타', '스웨그': '스웨그', '자전거': '자전거', '봄바람': '봄바람', '잠자기_전': '잠자기_전', '장르Top100': '장르top100', '홈트레이닝': '홈트레이닝', 'aomg': 'aomg', '8090세대': '8090세대', 'TR808': 'tr808', '루프탑': '루프탑', '기분좋은': '기분좋은', '대화': '대화', '생각나는': '생각나는', '피크닉': '피크닉', '운동': '운동', '외힙입문': '외힙입문', '스테디셀러': '스테디셀러', '노래방': '노래방', '리듬앤블루스': '리듬앤블루스', '음색부자': '음색부자', '추천노래': '추천노래', '장르불문': '장르불문', '프랑스': '프랑스', '독서': '독서', '도입부부터좋은'

## 특수문자 제거

In [214]:
import re

def make_dict_special(tag_set):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 아래를 제외한 문자는 ''로 대체한다
            # \uAC00-WuD7A3 : 모든 한글 음절
            # 0-9 : 숫자
            # a-zA-Z : 모든 영어
            tag_dict[i] = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z]', '', i)
    
    return tag_dict

In [215]:
tag_special_dict = make_dict_special(tag_set)

# 현재 태그들은 특수문자가 '_'만 존재한다
print(tag_special_dict)

{'61회': '61회', 'KPOP탐구생활': 'KPOP탐구생활', '조용한': '조용한', '플레이리스트': '플레이리스트', '인디뮤직': '인디뮤직', '찬양': '찬양', '잔잔한노래': '잔잔한노래', '운전': '운전', '산책이나': '산책이나', '방탄소년단': '방탄소년단', '팝': '팝', '7080': '7080', '도시의밤': '도시의밤', '밤에듣기좋은노래': '밤에듣기좋은노래', 'hilite': 'hilite', '무한도전': '무한도전', '꿀보이스': '꿀보이스', '도시': '도시', '베르디': '베르디', '90년대': '90년대', '게임': '게임', '차트': '차트', '집콕': '집콕', '하트시그널3': '하트시그널3', '아재': '아재', '비': '비', '홍연언니': '홍연언니', '편집샵': '편집샵', '가사없는': '가사없는', '외로운': '외로운', '기분을': '기분을', '피쉬슈즈': '피쉬슈즈', '중독성': '중독성', '한국ost': '한국ost', '일본': '일본', '화이팅': '화이팅', '여성': '여성', '외로움': '외로움', '5살': '5살', '여가수가많타': '여가수가많타', '스웨그': '스웨그', '자전거': '자전거', '봄바람': '봄바람', '잠자기_전': '잠자기전', '장르Top100': '장르Top100', '홈트레이닝': '홈트레이닝', 'aomg': 'aomg', '8090세대': '8090세대', 'TR808': 'TR808', '루프탑': '루프탑', '기분좋은': '기분좋은', '대화': '대화', '생각나는': '생각나는', '피크닉': '피크닉', '운동': '운동', '외힙입문': '외힙입문', '스테디셀러': '스테디셀러', '노래방': '노래방', '리듬앤블루스': '리듬앤블루스', '음색부자': '음색부자', '추천노래': '추천노래', '장르불문': '장르불문', '프랑스': '프랑스', '독서': '독서', '도입부부터좋은':

## Stopword 제거

- 현재 태그는 단순 단어나 띄어쓰기가 제거된 단어의 조합으로 구성
- 임의의 Stopword를 선정할 필요가 존재
- 현재 아래 코드는 보류

In [216]:
# 설정 필요
stopwords = {'날려버려'}

In [217]:
def make_dict_stop(tag_set, stopwords):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            # 빈 칸으로 만든 뒤 다른 함수로 제거할 예정
            if i in stopwords:
                tag_dict[i] = ''
            else:
                tag_dict[i] = i
                
    return tag_dict

In [218]:
tag_stop_dict = make_dict_stop(tag_set, stopwords)
print(tag_stop_dict)

{'61회': '61회', 'KPOP탐구생활': 'KPOP탐구생활', '조용한': '조용한', '플레이리스트': '플레이리스트', '인디뮤직': '인디뮤직', '찬양': '찬양', '잔잔한노래': '잔잔한노래', '운전': '운전', '산책이나': '산책이나', '방탄소년단': '방탄소년단', '팝': '팝', '7080': '7080', '도시의밤': '도시의밤', '밤에듣기좋은노래': '밤에듣기좋은노래', 'hilite': 'hilite', '무한도전': '무한도전', '꿀보이스': '꿀보이스', '도시': '도시', '베르디': '베르디', '90년대': '90년대', '게임': '게임', '차트': '차트', '집콕': '집콕', '하트시그널3': '하트시그널3', '아재': '아재', '비': '비', '홍연언니': '홍연언니', '편집샵': '편집샵', '가사없는': '가사없는', '외로운': '외로운', '기분을': '기분을', '피쉬슈즈': '피쉬슈즈', '중독성': '중독성', '한국ost': '한국ost', '일본': '일본', '화이팅': '화이팅', '여성': '여성', '외로움': '외로움', '5살': '5살', '여가수가많타': '여가수가많타', '스웨그': '스웨그', '자전거': '자전거', '봄바람': '봄바람', '잠자기_전': '잠자기_전', '장르Top100': '장르Top100', '홈트레이닝': '홈트레이닝', 'aomg': 'aomg', '8090세대': '8090세대', 'TR808': 'TR808', '루프탑': '루프탑', '기분좋은': '기분좋은', '대화': '대화', '생각나는': '생각나는', '피크닉': '피크닉', '운동': '운동', '외힙입문': '외힙입문', '스테디셀러': '스테디셀러', '노래방': '노래방', '리듬앤블루스': '리듬앤블루스', '음색부자': '음색부자', '추천노래': '추천노래', '장르불문': '장르불문', '프랑스': '프랑스', '독서': '독서', '도입부부터좋은'

## Stemming 제거

- 한글에서 동일한 의미여도 표현이 다른 경우를 자동적으로 찾는 것은 불가능하다
- 임의로 동일한 의미의 단어 묶음을 만들 필요가 있다
- 현재 아래 코드는 보류

In [219]:
# 설정 필요
stemmings = {
    '락' : {'락', '록'}
}

In [220]:
def make_dict_stem(tag_set, stemmings):
    tag_dict = dict()
    
    for i in tag_set:
        if not i in tag_dict:
            
            # stemming으로 설정한 단어가 존재하는지 확인
            for j in stemmings:
                if i in stemmings[j]:
                    tag_dict[i] = j
                    break
            
            # 존재하지 않는 경우
            if not i in tag_dict:
                tag_dict[i] = i
    
    return tag_dict

In [221]:
tag_stem_dict = make_dict_stem(tag_set, stemmings)
print(tag_stem_dict)

{'61회': '61회', 'KPOP탐구생활': 'KPOP탐구생활', '조용한': '조용한', '플레이리스트': '플레이리스트', '인디뮤직': '인디뮤직', '찬양': '찬양', '잔잔한노래': '잔잔한노래', '운전': '운전', '산책이나': '산책이나', '방탄소년단': '방탄소년단', '팝': '팝', '7080': '7080', '도시의밤': '도시의밤', '밤에듣기좋은노래': '밤에듣기좋은노래', 'hilite': 'hilite', '무한도전': '무한도전', '꿀보이스': '꿀보이스', '도시': '도시', '베르디': '베르디', '90년대': '90년대', '게임': '게임', '차트': '차트', '집콕': '집콕', '하트시그널3': '하트시그널3', '아재': '아재', '비': '비', '홍연언니': '홍연언니', '편집샵': '편집샵', '가사없는': '가사없는', '외로운': '외로운', '기분을': '기분을', '피쉬슈즈': '피쉬슈즈', '중독성': '중독성', '한국ost': '한국ost', '일본': '일본', '화이팅': '화이팅', '여성': '여성', '외로움': '외로움', '5살': '5살', '여가수가많타': '여가수가많타', '스웨그': '스웨그', '자전거': '자전거', '봄바람': '봄바람', '잠자기_전': '잠자기_전', '장르Top100': '장르Top100', '홈트레이닝': '홈트레이닝', 'aomg': 'aomg', '8090세대': '8090세대', 'TR808': 'TR808', '루프탑': '루프탑', '기분좋은': '기분좋은', '대화': '대화', '생각나는': '생각나는', '피크닉': '피크닉', '운동': '운동', '외힙입문': '외힙입문', '스테디셀러': '스테디셀러', '노래방': '노래방', '리듬앤블루스': '리듬앤블루스', '음색부자': '음색부자', '추천노래': '추천노래', '장르불문': '장르불문', '프랑스': '프랑스', '독서': '독서', '도입부부터좋은'

## 태그 dict 적용 함수

In [222]:
def apply_tag_dict(data, tag_dict):
    apply_data = data.copy()
    
    for i in range(len(apply_data)):
        tag = apply_data['tags'][i]
        apply_tag = []
        
        for j in tag:
            if not tag_dict[j] in apply_tag and tag_dict[j] != "":
                apply_tag.append(tag_dict[j])
        
        apply_data['tags'][i] = apply_tag

    return apply_data

In [223]:
stop_data = apply_tag_dict(train_data_sample2, tag_stop_dict)
stop_data.head()

C:\Users\pigwo\AppData\Local\Temp\ipykernel_1872\2377676666.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apply_data['tags'][i] = apply_tag


,tags,song_id,like_cnt
0,"[빌로우, EDMFloor]","[582347, 577988, 4096, 223252, 108282, 279668,...",0
1,"[기분전환, 밤, 새벽]","[275780, 507372, 515378, 69298, 485934, 70353,...",11
2,[댄스],"[569611, 526963, 153766, 530329, 259795, 38108...",9
3,"[걸그룹, 발라드]","[250038, 242240, 566455, 462740, 51643, 148007...",8
4,"[휴식, 가을, 잔잔한, 귀르가즘, 우울, 발라드, soul, JAZZY, 감성, ...","[582800, 450067, 500795, 204042, 350054, 56196...",7


In [224]:
# stopword가 제거됨
stop_tag_set = make_tag_set(stop_data)
print(stop_tag_set)

{'61회', 'KPOP탐구생활', '조용한', '플레이리스트', '인디뮤직', '찬양', '잔잔한노래', '운전', '산책이나', '방탄소년단', '팝', '7080', '도시의밤', '밤에듣기좋은노래', 'hilite', '무한도전', '꿀보이스', '도시', '베르디', '90년대', '게임', '차트', '집콕', '하트시그널3', '아재', '비', '홍연언니', '편집샵', '가사없는', '외로운', '기분을', '피쉬슈즈', '중독성', '한국ost', '일본', '화이팅', '여성', '외로움', '5살', '여가수가많타', '스웨그', '자전거', '봄바람', '잠자기_전', '장르Top100', '홈트레이닝', 'aomg', '8090세대', 'TR808', '루프탑', '기분좋은', '대화', '생각나는', '피크닉', '운동', '외힙입문', '스테디셀러', '노래방', '리듬앤블루스', '음색부자', '추천노래', '장르불문', '프랑스', '독서', '도입부부터좋은', 'Motown', '응답하라', '팝발라드', '벚꽃', '잔잔하고_따스한', '아이돌', '어번컨템포러리', '아픔', 'TOP100', '트로트Top100', '걸그룹', '아이돌탐구생활', '잔잔한_노래', '선선한밤', '세련된', '새벽', '썸', '성인가요Top100', '미국', '국내힙합', '힙합클럽', '혼자', '러블리쏘', '노을', '썸녀', '머리를맑게해주는', '왜그럴까', 'composer', '부드럽고', 'OST', '4살', '연주곡', '가을감성', '일본노래추천', '빌보드Hot50', '성탄절', '나만의Best3', '달콤', '도입부', '비트', '원곡', '인기곡', '헤어진후', '그리움', '식케이', '공감', '위로', 'Pop', '겨울공기', 'Rock', '겨울감성', '감미로움', '명상', '신청곡', '아기클래식', '좋은노래', '많이들어본', '데이트', '국내', '아기동화', '운전할때', '노동요

## 태그 불균형 데이터

## 불균형 정도 측정

- 한 태그를 가지는 음악의 최대 개수, 한 태그를 가지는 음악의 개수를 추출
- 가중치 부여에 관해서는 논의가 필요

In [225]:
def make_song_num_dict(data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(data)):
        songs = data['song_id'][i]
        tags = data['tags'][i]
        
        for j in tags:
            if not j in song_ids:
                song_ids[j] = set(songs)
            
            else:
                song_ids[j].update(songs)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

In [226]:
song_num_dict, song_num_max = make_song_num_dict(train_data_sample2)
print(song_num_max)
print(song_num_dict)

3189
{'빌로우': 42, 'EDMFloor': 42, '기분전환': 3189, '밤': 1392, '새벽': 1713, '댄스': 870, '걸그룹': 328, '발라드': 2052, '휴식': 1919, '가을': 617, '잔잔한': 2099, '귀르가즘': 16, '우울': 213, 'soul': 113, 'JAZZY': 16, '감성': 1569, 'groove': 16, 'OST': 746, '신곡': 32, '박자': 32, '비트': 92, '재즈': 657, '월요병': 111, '좋은노래': 17, '음색': 116, '힐링': 1576, '편안한': 173, '겨울': 687, '저녁': 476, '매장음악': 1028, '겨울감성': 159, '카페': 1166, '드라이브': 2526, '사랑': 1764, '피크닉': 26, '오후': 226, '여행': 798, '운동': 580, '락': 693, '무도': 6, '무한도전': 6, '카페음악': 51, '커피와': 17, '봄': 352, '벚꽃': 17, '기분좋은': 85, 'Pop': 490, '산책': 514, '힙합': 1492, 'label_crew': 33, '랩': 918, '국내힙합': 64, '트렌드': 33, '쇼미더머니': 33, '스웨그': 33, 'aomg': 51, 'hilite': 33, '2010': 45, '게임': 12, '넥슨': 12, '니드포스피드': 12, 'needforspeed': 12, '니드포스피드엣지': 12, '연주곡': 325, '피아노연주곡모음': 178, '피아노': 349, '카페에서듣기좋은음악': 178, '아메리카노': 178, '가사없는': 178, '뉴에이지피아노연주곡': 178, '머리를맑게해주는': 178, '아침잠을깨워주는': 178, '안녕': 20, '집': 116, '위로': 291, 'Home': 20, '스트레스': 578, '집중': 208, '느낌있는': 335, '여유': 147, '회상': 

## 장르 데이터

## 불균형 데이터 처리

In [227]:
def make_genre_num_dict(appended_data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(appended_data)):
        song = appended_data['song_id'][i]
        gnrs = appended_data['gnr'][i]
        
        for j in gnrs:
            if not j in song_ids:
                song_ids[j] = {song}
            
            else:
                song_ids[j].add(song)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

In [228]:
genre_num_dict, genre_num_max = make_genre_num_dict(song_tag_appended)
print(genre_num_max)
print(genre_num_dict)

3313
{'GN1104': 529, 'GN1103': 338, 'GN1101': 1085, 'GN1014': 44, 'GN1001': 986, 'GN2701': 81, 'GN2702': 38, 'GN2703': 78, 'GN0904': 29, 'GN2704': 317, 'GN1109': 111, 'GN1102': 435, 'GN1301': 665, 'GN1302': 537, 'GN0901': 1833, 'GN1201': 773, 'GN1202': 200, 'GN1209': 210, 'GN1105': 127, 'GN1503': 148, 'GN1501': 1353, 'GN1807': 79, 'GN1801': 593, 'GN1003': 120, 'GN1013': 298, 'GN0902': 383, 'GN2502': 714, 'GN0401': 1094, 'GN2501': 1478, 'GN0205': 675, 'GN2506': 790, 'GN0201': 1260, 'GN0105': 1875, 'GN0101': 3313, 'GN2505': 348, 'GN0303': 1033, 'GN2504': 202, 'GN0301': 1767, 'GN0403': 768, 'GN2503': 741, 'GN0104': 739, 'GN0402': 331, 'GN0805': 462, 'GN0801': 684, 'GN1308': 107, 'GN1304': 215, 'GN0908': 248, 'GN1504': 906, 'GN0302': 215, 'GN1701': 702, 'GN1702': 150, 'GN1403': 7, 'GN1401': 136, 'GN1710': 88, 'GN1402': 87, 'GN0508': 128, 'GN0804': 37, 'GN0502': 484, 'GN0501': 1829, 'GN1704': 27, 'GN1703': 138, 'GN1708': 13, 'GN1502': 102, 'GN1306': 21, 'GN1608': 85, 'GN1601': 486, 'GN0509'